# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('orders.zip')
orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
orders_agg = orders.groupby(by=['CustomerID','Country']).agg({'amount_spent':'sum'})
orders_agg.head(5)

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


In [4]:
# label the customers by the referred quantiles of spent
orders_agg.loc[orders_agg['amount_spent'] >= orders_agg['amount_spent'].quantile(0.75), 'label'] = 'Preferred'
orders_agg.loc[orders_agg['amount_spent'] >= orders_agg['amount_spent'].quantile(0.95), 'label'] = 'VIP'
orders_agg.head(5)

,,amount_spent,label
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,NaN


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [5]:
orders_label = orders_agg
orders_label['VIP'] = np.where(orders_label['label'] == 'VIP', 1, 0)
orders_label.head(5)

,,amount_spent,label,VIP
CustomerID,Country,,,
12346,United Kingdom,77183.60,VIP,1
12347,Iceland,4310.00,Preferred,0
12348,Finland,1797.24,Preferred,0
12349,Italy,1757.55,Preferred,0
12350,Norway,334.40,NaN,0


In [6]:
orders_final = orders_label.groupby(by=['Country']).agg({'VIP':'sum'})
orders_final.sort_values(by='VIP', ascending=False)[:5]

,VIP
Country,
United Kingdom,178
Germany,11
France,9
Switzerland,3
Spain,2


In [ ]:
# The country with more VIP customers is United Kingdom

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [7]:
orders_label['Preferred'] = np.where(orders_label['label'] == 'Preferred', 1, 0)
orders_label.head(5)

,,amount_spent,label,VIP,Preferred
CustomerID,Country,,,,
12346,United Kingdom,77183.60,VIP,1,0
12347,Iceland,4310.00,Preferred,0,1
12348,Finland,1797.24,Preferred,0,1
12349,Italy,1757.55,Preferred,0,1
12350,Norway,334.40,NaN,0,0


In [10]:
orders_label['VIP+Preferred'] = orders_label['VIP'] + orders_label['Preferred']
orders_label.head(5)

,,amount_spent,label,VIP,Preferred,VIP+Preferred
CustomerID,Country,,,,,
12346,United Kingdom,77183.60,VIP,1,0,1
12347,Iceland,4310.00,Preferred,0,1,1
12348,Finland,1797.24,Preferred,0,1,1
12349,Italy,1757.55,Preferred,0,1,1
12350,Norway,334.40,NaN,0,0,0


In [12]:
orders_final2 = orders_label.groupby(by=['Country']).agg({'VIP+Preferred':'sum'})
orders_final2.sort_values(by='VIP+Preferred', ascending=False)[:5]

,VIP+Preferred
Country,
United Kingdom,934
Germany,39
France,29
Belgium,11
Switzerland,9


In [ ]:
# The country with more VIP+Preferred customers combined is still United Kingdom